In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Step 1: Import the necessary library
from google.colab import drive

# Step 2: Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load datasets from Google Drive
train_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/Train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/Test.csv')
train_gap_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/Gap_Train.csv')
test_gap_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/Gap_Test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/SampleSubmission.csv')

In [ ]:
train_df.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,P,K,Ca,Mg,S,Fe,Mn,Zn,Cu,B
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,...,0.34,147,6830,2310,5.66,75.2,85.0,0.82,2.98,0.24
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,...,11.70,151,1180,235,19.40,96.2,409.0,2.57,4.32,0.10
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,...,21.80,151,1890,344,11.00,76.7,65.0,1.95,1.24,0.22
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,...,39.90,201,6660,719,14.90,81.9,73.0,4.90,3.08,0.87
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,...,1.00,90,7340,1160,8.66,69.4,149.0,0.55,3.03,0.31


In [ ]:
test_df.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,para,parv,ph20,slope,snd20,soc20,tim,wp,xhp20,BulkDensity
0,site_id_hgJpkz,ID_NGS9Bx,69.170794,44.522885,6.86,144,256,910,108,186,...,37.940418,467.619293,6.825,1.056416,25.50,15.25,8.732471,0.016981,0.005831,1.20
1,site_id_olmuI5,ID_YdVKXw,68.885265,44.741057,7.08,129,260,851,110,187,...,35.961353,542.590149,6.725,0.730379,18.75,14.00,10.565657,0.021030,0.005134,1.24
2,site_id_PTZdJz,ID_MZAlfE,68.970210,44.675777,6.50,142,259,901,109,187,...,38.983898,416.385437,6.825,1.146542,21.00,14.00,9.590125,0.018507,0.004480,1.23
3,site_id_DOTgr8,ID_GwCCMN,69.068751,44.647707,6.82,142,261,847,109,187,...,39.948471,374.971008,6.725,0.567210,23.25,12.25,9.669279,0.021688,0.006803,1.22
4,site_id_1rQNvy,ID_K8sowf,68.990002,44.577607,6.52,145,253,1109,110,186,...,33.658615,361.233643,6.200,1.169207,26.25,18.25,7.895920,0.023016,0.000874,1.23


In [ ]:
train_gap_df.head()

,Nutrient,Required,Available,Gap,PID
0,N,100.0,3796.0000,-3696.0000,ID_I5RGjv
1,P,40.0,0.9928,39.0072,ID_I5RGjv
2,K,52.0,429.2400,-377.2400,ID_I5RGjv
3,Ca,12.0,19943.6000,-19931.6000,ID_I5RGjv
4,Mg,8.0,6745.2000,-6737.2000,ID_I5RGjv


In [ ]:
test_gap_df = pd.merge(test_gap_df, test_df[['PID', 'BulkDensity']], on='PID', how='left')

In [ ]:
test_gap_df.head()

,Nutrient,Required,PID,BulkDensity
0,N,100.0,ID_NGS9Bx,1.2
1,P,40.0,ID_NGS9Bx,1.2
2,K,52.0,ID_NGS9Bx,1.2
3,Ca,12.0,ID_NGS9Bx,1.2
4,Mg,8.0,ID_NGS9Bx,1.2


In [ ]:
sample_submission.head()

,ID,Gap
0,ID_002W8m_B,0
1,ID_002W8m_Ca,0
2,ID_002W8m_Cu,0
3,ID_002W8m_Fe,0
4,ID_002W8m_K,0


In [ ]:
# Fill missing values with the mean for columns with missing values in train_df
for column in train_df.columns:
  if train_df[column].isnull().any():
    train_df[column].fillna(train_df[column].mean(), inplace=True)

# Fill missing values with the mean for columns with missing values in test_df
for column in test_df.columns:
  if test_df[column].isnull().any():
    test_df[column].fillna(test_df[column].mean(), inplace=True)

In [ ]:
train_df.columns

Index(['site', 'PID', 'lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15',
       'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn',
       'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope',
       'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity', 'N', 'P', 'K',
       'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B'],
      dtype='object')

In [ ]:
!pip install pykrige

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.6/979.6 kB 16.7 MB/s eta 0:00:00


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykrige import variogram_models
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import itertools

# List of independent variables to analyze
independent_vars = [
    'pH', 'alb', 'bio1', 'bio12', 'bio15', 'bio7',
    'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls',
    'lstd', 'lstn', 'mb1', 'mb2', 'mb3', 'mb7', 'mdem',
    'para', 'parv', 'ph20', 'slope', 'snd20', 'soc20',
    'tim', 'wp', 'xhp20', 'BulkDensity'
]

def calculate_empirical_variogram(df, variable, max_dist=None, n_lags=20):
    """
    Calculate empirical variogram for a given variable

    Parameters:
    -----------
    df : pandas DataFrame
        Dataframe containing coordinates and the variable
    variable : str
        Column name of the variable to analyze
    max_dist : float, optional
        Maximum distance to consider for variogram
    n_lags : int, optional
        Number of distance lags for binning

    Returns:
    --------
    lags : numpy array
        Lag distances (bin centers)
    semivariance : numpy array
        Semivariance values for each lag
    counts : numpy array
        Number of point pairs in each lag bin
    """
    # Extract coordinates and values
    coords = df[['lon', 'lat']].values
    values = df[variable].values

    n_points = len(coords)

    # Calculate all pairwise distances and value differences
    distances = np.zeros((n_points * (n_points - 1)) // 2)
    value_diffs = np.zeros((n_points * (n_points - 1)) // 2)

    idx = 0
    for i in range(n_points):
        for j in range(i+1, n_points):
            # Euclidean distance
            dist = np.sqrt(((coords[i] - coords[j])**2).sum())
            distances[idx] = dist

            # Squared difference in values
            value_diff = (values[i] - values[j])**2
            value_diffs[idx] = value_diff

            idx += 1

    # Determine maximum distance if not provided
    if max_dist is None:
        max_dist = np.percentile(distances, 75)  # Use 75th percentile as default

    # Create distance bins
    bins = np.linspace(0, max_dist, n_lags + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    # Assign point pairs to bins
    bin_indices = np.digitize(distances, bins) - 1

    # Mask out pairs beyond max_dist
    valid_indices = bin_indices < n_lags

    # Calculate semivariance for each bin
    semivariance = np.zeros(n_lags)
    counts = np.zeros(n_lags, dtype=int)

    for i in range(n_lags):
        bin_mask = bin_indices == i
        if np.any(bin_mask):
            semivariance[i] = np.mean(value_diffs[bin_mask]) / 2  # Divide by 2 for semivariance
            counts[i] = np.sum(bin_mask)

    return bin_centers, semivariance, counts

def fit_variogram_model(lags, semivariance, model_type='spherical'):
    """
    Fit a theoretical variogram model to empirical data

    Parameters:
    -----------
    lags : numpy array
        Lag distances
    semivariance : numpy array
        Semivariance values for each lag
    model_type : str
        Type of variogram model ('spherical', 'exponential', 'gaussian', 'linear')

    Returns:
    --------
    parameters : tuple
        Fitted parameters (sill, range, nugget)
    fitted_values : numpy array
        Fitted semivariance values
    rmse : float
        Root mean square error of the fit
    """
    # Remove NaN values
    mask = ~np.isnan(semivariance)
    lags_clean = lags[mask]
    semivar_clean = semivariance[mask]

    if len(lags_clean) < 3:
        return None, None, None

    # Initial parameter estimates
    initial_sill = np.nanmax(semivar_clean)
    initial_range = np.nanmean(lags_clean)
    initial_nugget = np.nanmin(semivar_clean)

    # Avoid negative values
    initial_sill = max(initial_sill, 0.001)
    initial_range = max(initial_range, 0.001)
    initial_nugget = max(initial_nugget, 0)

    # Bounds for parameters (sill, range, nugget)
    bounds = [(0.001, initial_sill*3), (0.001, max(lags_clean)*2), (0, initial_sill)]

    # Select variogram model function
    if model_type == 'spherical':
        model_func = variogram_models.spherical_variogram_model
    elif model_type == 'exponential':
        model_func = variogram_models.exponential_variogram_model
    elif model_type == 'gaussian':
        model_func = variogram_models.gaussian_variogram_model
    elif model_type == 'linear':
        model_func = variogram_models.linear_variogram_model
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    # Optimization function
    def objective(params):
        sill, range_param, nugget = params
        fitted = model_func(params, lags_clean)
        return np.sum((semivar_clean - fitted)**2)

    # Simple grid search to find good parameters
    best_params = None
    best_error = float('inf')

    sill_values = np.linspace(initial_sill*0.5, initial_sill*1.5, 5)
    range_values = np.linspace(initial_range*0.5, initial_range*1.5, 5)
    nugget_values = np.linspace(0, initial_sill*0.5, 5)

    for sill, range_param, nugget in itertools.product(sill_values, range_values, nugget_values):
        params = (sill, range_param, nugget)
        try:
            fitted = model_func(params, lags_clean)
            error = np.sum((semivar_clean - fitted)**2)
            if error < best_error:
                best_error = error
                best_params = params
        except:
            continue

    if best_params is None:
        return None, None, None

    # Calculate fitted values and RMSE
    fitted_values = model_func(best_params, lags)
    rmse = np.sqrt(mean_squared_error(semivar_clean, model_func(best_params, lags_clean)))

    return best_params, fitted_values, rmse

def plot_variogram(lags, semivariance, counts, fitted_values=None, params=None, title="Empirical Variogram"):
    """
    Plot empirical variogram and fitted model
    """
    plt.figure(figsize=(10, 6))

    # Plot empirical variogram
    plt.scatter(lags, semivariance, s=counts/np.max(counts)*100, alpha=0.7, c='blue', label='Empirical')

    # Plot fitted model if available
    if fitted_values is not None and params is not None:
        plt.plot(lags, fitted_values, 'r-', label=f'Fitted model (sill={params[0]:.3f}, range={params[1]:.3f}, nugget={params[2]:.3f})')

    plt.xlabel('Lag Distance')
    plt.ylabel('Semivariance')
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend()

    return plt.gcf()

def variogram_analysis(train_df, test_df, var_list=None):
    """
    Perform variogram analysis on variables

    Parameters:
    -----------
    train_df : pandas DataFrame
        Training dataset
    test_df : pandas DataFrame
        Test dataset
    var_list : list, optional
        List of variables to analyze

    Returns:
    --------
    train_with_variogram : pandas DataFrame
        Training data augmented with variogram-based features
    test_with_variogram : pandas DataFrame
        Test data augmented with variogram-based features
    variogram_results : dict
        Dictionary containing variogram analysis results
    """
    if var_list is None:
        var_list = independent_vars

    # Step 1: Add dataset identifier
    train_df = train_df.copy()
    test_df = test_df.copy()
    train_df['dataset'] = 'train'
    test_df['dataset'] = 'test'

    # Step 2: Concatenate datasets
    combined_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

    # Scale variables to have comparable ranges
    scaler = StandardScaler()
    combined_scaled = combined_df.copy()
    for var in var_list:
        if var in combined_df.columns:
            combined_scaled[var] = scaler.fit_transform(combined_df[[var]])

    # Results dictionary
    variogram_results = {}

    # Create DataFrames for augmented features
    train_with_variogram = train_df.copy()
    test_with_variogram = test_df.copy()

    # Model types to try
    model_types = ['spherical', 'exponential', 'gaussian']

    # Analyze each variable
    for var in var_list:
        if var not in combined_df.columns:
            continue

        print(f"Processing variable: {var}")

        # Calculate empirical variogram
        lags, semivariance, counts = calculate_empirical_variogram(combined_scaled, var)

        # Find best model
        best_model = None
        best_rmse = float('inf')
        best_params = None
        best_fitted = None

        for model_type in model_types:
            params, fitted, rmse = fit_variogram_model(lags, semivariance, model_type)

            if params is not None and rmse < best_rmse:
                best_model = model_type
                best_rmse = rmse
                best_params = params
                best_fitted = fitted

        if best_params is None:
            print(f"  Could not fit variogram model for {var}")
            continue

        # Store results
        variogram_results[var] = {
            'lags': lags,
            'semivariance': semivariance,
            'counts': counts,
            'best_model': best_model,
            'parameters': best_params,
            'fitted_values': best_fitted,
            'rmse': best_rmse
        }

        # Extract variogram parameters
        sill, range_param, nugget = best_params

        # Add variogram parameters as features
        for df in [train_with_variogram, test_with_variogram]:
            df[f'{var}_sill'] = sill
            df[f'{var}_range'] = range_param
            df[f'{var}_nugget'] = nugget
            df[f'{var}_spatial_dependency'] = (sill - nugget) / sill if sill > 0 else 0

        # Calculate distance-based variogram values for each point
        for idx, row in combined_df.iterrows():
            point_coords = np.array([row['lon'], row['lat']])

            # Calculate distances to all other points
            coords = combined_df[['lon', 'lat']].values
            distances = np.sqrt(np.sum((coords - point_coords)**2, axis=1))

            # Exclude self
            mask = distances > 0

            if np.sum(mask) > 0:
                # Calculate variogram-based weights (inverse of variogram value)
                if best_model == 'spherical':
                    variogram_values = variogram_models.spherical_variogram_model((sill, range_param, nugget), distances[mask])
                elif best_model == 'exponential':
                    variogram_values = variogram_models.exponential_variogram_model((sill, range_param, nugget), distances[mask])
                elif best_model == 'gaussian':
                    variogram_values = variogram_models.gaussian_variogram_model((sill, range_param, nugget), distances[mask])
                else:
                    variogram_values = variogram_models.linear_variogram_model((sill, range_param, nugget), distances[mask])

                # Weight by inverse of variogram value (closer points have higher weight)
                weights = 1 / (variogram_values + 1e-10)
                weights = weights / np.sum(weights)

                # Calculate weighted average of neighboring values
                var_values = combined_df.loc[mask, var].values
                weighted_avg = np.sum(weights * var_values)

                # Add as feature
                if idx < len(train_with_variogram):
                    train_with_variogram.loc[idx, f'{var}_variogram_avg'] = weighted_avg
                else:
                    test_with_variogram.loc[idx - len(train_with_variogram), f'{var}_variogram_avg'] = weighted_avg

    # Drop dataset column
    train_with_variogram = train_with_variogram.drop(columns=['dataset'])
    test_with_variogram = test_with_variogram.drop(columns=['dataset'])

    return train_with_variogram, test_with_variogram, variogram_results

def plot_all_variograms(variogram_results, save_dir=None):
    """
    Plot all variograms and save to files if save_dir is provided
    """
    for var, results in variogram_results.items():
        title = f"Variogram for {var}"
        fig = plot_variogram(
            results['lags'],
            results['semivariance'],
            results['counts'],
            results['fitted_values'],
            results['parameters'],
            title
        )

        if save_dir:
            import os
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            fig.savefig(os.path.join(save_dir, f"variogram_{var}.png"))
            plt.close(fig)
        else:
            plt.show()

# Main execution
def main(train_df, test_df):
    # Perform variogram analysis
    train_with_variogram, test_with_variogram, variogram_results = variogram_analysis(train_df, test_df)


    print(f"Added {len([col for col in train_with_variogram.columns if '_sill' in col])} variogram-based features")

    return train_with_variogram, test_with_variogram

# Execute main function with your dataframes
train_with_variogram, test_with_variogram = main(train_df, test_df)

Processing variable: pH
Processing variable: alb
Processing variable: bio1
Processing variable: bio12
Processing variable: bio15
Processing variable: bio7
Processing variable: bp
Processing variable: cec20
Processing variable: dows
Processing variable: ecec20
Processing variable: hp20
Processing variable: ls
Processing variable: lstd
Processing variable: lstn
Processing variable: mb1
Processing variable: mb2
Processing variable: mb3
Processing variable: mb7
Processing variable: mdem
Processing variable: para
Processing variable: parv
Processing variable: ph20
Processing variable: slope
Processing variable: snd20
Processing variable: soc20
Processing variable: tim
Processing variable: wp
Processing variable: xhp20
Processing variable: BulkDensity
Added 29 variogram-based features


In [ ]:
# Define the save path
save_path = '/content/drive/MyDrive/Amini Soil Prediction Challenge/'

# Save DataFrames as CSV files
train_with_variogram.to_csv(save_path + 'train_with_variogram.csv', index=False)
test_with_variogram.to_csv(save_path + 'test_with_variogram.csv', index=False)